In [4]:
import pandas as pd
Auto = pd.read_csv('Auto.csv', header=0, na_values='?')
#Auto

In [8]:
Auto = Auto.dropna().reset_index(drop=True)
Auto.shape
import numpy as np
np.random.seed(1)
train = np.random.choice(len(Auto), 196, replace=False)
select = np.in1d(range(len(Auto)), train)

In [29]:
import statsmodels.formula.api as smf
lm = smf.ols('mpg ~ horsepower', data = Auto[select]).fit()
##lm.summary()

In [13]:
preds=lm.predict(Auto)
squared_error=(Auto['mpg']-preds)**2
np.mean(squared_error[select])

24.62301015144334

In [30]:
np.mean(squared_error[~select])

19.63523068315503

In [31]:
lm2 = smf.ols('mpg ~ horsepower + I(horsepower ** 2.0)'
              ,data = Auto[select]).fit()
preds = lm2.predict(Auto)
squared_error=(Auto['mpg']-preds)**2
np.mean(squared_error[select])

18.352971128237044

In [32]:
np.mean(squared_error[~select])

19.722533470489978

In [28]:
np.random.seed(2)
train=np.random.choice(Auto.shape[0], 196, replace=False)
select = np.in1d(range(Auto.shape[0]), train)

In [37]:
x=pd.DataFrame(Auto.horsepower)
y=Auto.mpg
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x,y)
##model.intercept_
##model.coef_

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [35]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=len(x))
test=cross_val_score(model, x, y, cv=k_fold, scoring=
                     'neg_mean_squared_error', n_jobs=-1)
np.mean(-test)

24.23151351792923

In [40]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
MSE=[]
for porder in range(1, 6):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), 
                     ('linear', LinearRegression())])
    test=cross_val_score(model, x, y, cv=k_fold,
                         scoring='neg_mean_squared_error', n_jobs=-1)
    MSE.append(np.mean(-test))
MSE

[24.231513517929226,
 19.248213124489393,
 19.334984064131373,
 19.424430309564396,
 19.03321677574123]